In [6]:

"""

ля 151 406 договоров страхования транспортных средств известны значения ряда признаков, 
в том числе пол, возраст, стаж вождения и коэффициент бонус-малус водителя, тип, марка, модель, 
год выпуска, страна – производитель, мощность и объем двигателя, а также признак target, равный 1, 
если заключение договора с клиентом является рисковым, и 0 в противном случае (файл insclass_train.csv).

В обучающем наборе данных для каждого договора известны следующие поля:

variable_1 - агрегированный коэффициент бонус-малус (повышающий или понижающий стоимость полиса в зависимости от аварийности в предыдущие периоды);
variable_2 - индикатор расторжения договора по инициативе страхователя (клиента);
variable_3 - индикатор расторжения договора по инициативе страховщика (страховой компании);
variable_4 - идентификатор года выпуска транспортного средства;
variable_5 - идентификатор страны - производителя транспортного средства;
variable_6 - мощность двигателя в лошадиных силах;
variable_7 - объем двигателя в куб. см;
variable_8 - идентификатор стороны расположения руля (левый или правый);
variable_9 - пробег транспортного средства, покрываемый гарантией производителя;
variable_10 - индикатор действия гарантии на транспортное средство;
variable_11 - "мультидрайв" - индикатор допуска к управлению транспортным средством более одного водителя;
variable_12 - возраст транспортного средства (в мес.);
variable_13 - возраст водителя с максимальным стажем;
variable_14 - коэффициент возраст-стаж;
variable_15 - коэффициент краткосрочности;
variable_16 - коэффициент мощности;
variable_17 - коэффициент "мультидрайв";
variable_18 - территориальный коэффициент;
variable_19 - коэффициент "КНДР";
variable_20 - идентификатор канала продаж;
variable_21 - марка транспортного средства;
variable_22 - модель транспортного средства;
variable_23 - индикатор отечественных транспортных средств;
variable_24 - пол водителя с максимальным коэффициентом "возраст-стаж";
variable_25 - индикатор пролонгации;
variable_26 - индикатор совпадения собственника транспортного средства и водителя;
variable_27 - стаж водителя с максимальным коэффициентом "возраст-стаж";
variable_28 - тип транспортного средства;
target - класс риска, равный 1, если заключение договора с клиентом является рисковым, и 0 в противном случае.
В тестовом наборе данных каждому договору соответствует уникальный идентификатор id.
"""

'\n\nля 151 406 договоров страхования транспортных средств известны значения ряда признаков, \nв том числе пол, возраст, стаж вождения и коэффициент бонус-малус водителя, тип, марка, модель, \nгод выпуска, страна – производитель, мощность и объем двигателя, а также признак target, равный 1, \nесли заключение договора с клиентом является рисковым, и 0 в противном случае (файл insclass_train.csv).\n\nВ обучающем наборе данных для каждого договора известны следующие поля:\n\nvariable_1 - агрегированный коэффициент бонус-малус (повышающий или понижающий стоимость полиса в зависимости от аварийности в предыдущие периоды);\nvariable_2 - индикатор расторжения договора по инициативе страхователя (клиента);\nvariable_3 - индикатор расторжения договора по инициативе страховщика (страховой компании);\nvariable_4 - идентификатор года выпуска транспортного средства;\nvariable_5 - идентификатор страны - производителя транспортного средства;\nvariable_6 - мощность двигателя в лошадиных силах;\nvariab

In [11]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier

# =========================
# Загрузка данных
# =========================
TRAIN_PATH = "insclass_train.csv"
TEST_PATH  = "insclass_test.csv"

train = pd.read_csv(TRAIN_PATH)
test  = pd.read_csv(TEST_PATH)

y = train["target"].astype(int).values
X = train.drop(columns=["target"]).copy()
test_id = test["id"].values
X_test = test.drop(columns=["id"]).copy()

# =========================
# Препроцессинг
# =========================
drop_cols = []
for c in X.columns:
    if X[c].nunique(dropna=False) <= 1:
        drop_cols.append(c)

if X_test["variable_15"].isna().mean() > 0.999:
    drop_cols.append("variable_15")

drop_cols = sorted(set(drop_cols))
X.drop(columns=drop_cols, inplace=True, errors="ignore")
X_test.drop(columns=drop_cols, inplace=True, errors="ignore")

cat_cols = X.select_dtypes(include=["object"]).columns.tolist()
for c in cat_cols:
    X[c] = X[c].astype("string").fillna("__MISSING__")
    X_test[c] = X_test[c].astype("string").fillna("__MISSING__")

na_cols = [c for c in X.columns if X[c].isna().any()]
for c in na_cols:
    X[c + "__isna"] = X[c].isna().astype(np.uint8)
    X_test[c + "__isna"] = X_test[c].isna().astype(np.uint8)

cat_cols2 = [c for c in X.columns if X[c].dtype == "string" or X[c].dtype == "object"]
cat_idx = [X.columns.get_loc(c) for c in cat_cols2]

# =========================
# Лучшие параметры (без random_seed - будем менять)
# =========================
best_params = dict(
    loss_function="Logloss",
    eval_metric="AUC",
    bootstrap_type="MVS",
    grow_policy="Lossguide",
    iterations=6740,
    learning_rate=0.035,
    depth=5,
    l2_leaf_reg=2.25,
    random_strength=4.8,
    border_count=205,
    min_data_in_leaf=61,
    rsm=0.95,
    one_hot_max_size=13,
    max_ctr_complexity=6,
    auto_class_weights="SqrtBalanced",
    max_leaves=52,
    subsample=0.7,
    verbose=200,
)

# =========================
# Seed Averaging
# =========================
seeds = [7896,99999999, 568]
all_oof = []
all_test_preds = []

for seed in seeds:
    print(f"\n{'='*50}")
    print(f"Training with seed={seed}")
    print(f"{'='*50}")
    
    # Устанавливаем seed
    best_params['random_seed'] = seed
    
    # CV с этим сидом
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    
    oof = np.zeros(len(X), dtype=float)
    test_pred = np.zeros(len(X_test), dtype=float)
    
    for fold, (tr_idx, va_idx) in enumerate(skf.split(X, y), 1):
        print(f"\n--- Seed {seed}, Fold {fold}/5 ---")
        
        X_tr, y_tr = X.iloc[tr_idx], y[tr_idx]
        X_va, y_va = X.iloc[va_idx], y[va_idx]
        
        model = CatBoostClassifier(**best_params)
        model.fit(
            X_tr, y_tr,
            eval_set=(X_va, y_va),
            cat_features=cat_idx,
            use_best_model=True,
            early_stopping_rounds=235,
        )
        
        oof[va_idx] = model.predict_proba(X_va)[:, 1]
        test_pred += model.predict_proba(X_test)[:, 1] / 5
    
    # Сохраняем результаты этого сида
    oof_auc = roc_auc_score(y, oof)
    print(f"\nSeed {seed} OOF AUC: {oof_auc:.6f}")
    
    all_oof.append(oof)
    all_test_preds.append(test_pred)

# =========================
# Усреднение предсказаний
# =========================
print(f"\n{'='*50}")
print("AVERAGING RESULTS")
print(f"{'='*50}")

# Усредняем OOF
avg_oof = np.mean(all_oof, axis=0)
avg_oof_auc = roc_auc_score(y, avg_oof)
print(f"Average OOF AUC: {avg_oof_auc:.6f}")

# Усредняем тестовые предсказания
avg_test_pred = np.mean(all_test_preds, axis=0)

# =========================
# Подбор порога
# =========================
thr_grid = np.linspace(0.01, 0.99, 199)
best_thr, best_score = None, -1

for t in thr_grid:
    pred_bin = (avg_oof >= t).astype(int)
    s = roc_auc_score(y, pred_bin)
    if s > best_score:
        best_score = s
        best_thr = t

print(f"Best threshold: {best_thr:.4f}")
print(f"OOF AUC (binary): {best_score:.6f}")

# =========================
# Создание submission
# =========================
test_bin = (avg_test_pred >= best_thr).astype(int)
sub = pd.DataFrame({"id": test_id, "target": test_bin})
sub.to_csv("submission_11.csv", index=False)

print(f"\nSubmission saved: submission_11.csv")
print(f"Target distribution: {sub['target'].value_counts().to_dict()}")
print(sub.head(10))

np.save("test_pred_cb.npy", test_pred)
np.save("oof_cb.npy", oof)




Training with seed=7896

--- Seed 7896, Fold 1/5 ---
0:	test: 0.6003138	best: 0.6003138 (0)	total: 62.4ms	remaining: 7m
200:	test: 0.7351693	best: 0.7351693 (200)	total: 17.3s	remaining: 9m 22s
400:	test: 0.7392072	best: 0.7392072 (400)	total: 29.6s	remaining: 7m 47s
600:	test: 0.7397918	best: 0.7409963 (483)	total: 43.4s	remaining: 7m 23s
Stopped by overfitting detector  (235 iterations wait)

bestTest = 0.7409963253
bestIteration = 483

Shrink model to first 484 iterations.

--- Seed 7896, Fold 2/5 ---
0:	test: 0.5665814	best: 0.5665814 (0)	total: 61.9ms	remaining: 6m 57s
200:	test: 0.7351593	best: 0.7351673 (199)	total: 17.7s	remaining: 9m 35s
400:	test: 0.7403554	best: 0.7403554 (400)	total: 30s	remaining: 7m 54s
600:	test: 0.7429553	best: 0.7430516 (572)	total: 42.3s	remaining: 7m 11s
800:	test: 0.7436033	best: 0.7440357 (744)	total: 54.4s	remaining: 6m 43s
Stopped by overfitting detector  (235 iterations wait)

bestTest = 0.7440356691
bestIteration = 744

Shrink model to first 7

KeyboardInterrupt: 

In [10]:
import pandas as pd
from functools import reduce

paths = [
    "score_068163.csv",
    "score_068165.csv",
]

dfs = []
for i, p in enumerate(paths):
    df = pd.read_csv(p)
    df = df.rename(columns={"target": f"t{i}"})
    dfs.append(df)

m = reduce(lambda a,b: a.merge(b, on="id", how="inner"), dfs)

tcols = [c for c in m.columns if c.startswith("t")]
vote_sum = m[tcols].sum(axis=1)

# soft-vote (лучше для AUC)
m_out = pd.DataFrame({"id": m["id"], "target": vote_sum / len(tcols)})

# majority-vote (0/1)
m_out2 = pd.DataFrame({"id": m["id"], "target": (vote_sum >= (len(tcols)//2 + 1)).astype(int)})
m_out2.to_csv("ensemble_majority.csv", index=False)



In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier

# =========================
# Загрузка данных
# =========================
TRAIN_PATH = "insclass_train.csv"
TEST_PATH  = "insclass_test.csv"
PSEUDO_PATH = "score_068192.csv"  # лучший submission

train = pd.read_csv(TRAIN_PATH)
test  = pd.read_csv(TEST_PATH)
pseudo_sub = pd.read_csv(PSEUDO_PATH)

y = train["target"].astype(int).values
X = train.drop(columns=["target"]).copy()
test_id = test["id"].values
X_test = test.drop(columns=["id"]).copy()

# =========================
# Препроцессинг (как раньше)
# =========================
drop_cols = []
for c in X.columns:
    if X[c].nunique(dropna=False) <= 1:
        drop_cols.append(c)

if X_test["variable_15"].isna().mean() > 0.999:
    drop_cols.append("variable_15")

drop_cols = sorted(set(drop_cols))
X.drop(columns=drop_cols, inplace=True, errors="ignore")
X_test.drop(columns=drop_cols, inplace=True, errors="ignore")

cat_cols = X.select_dtypes(include=["object"]).columns.tolist()
for c in cat_cols:
    X[c] = X[c].astype("string").fillna("__MISSING__")
    X_test[c] = X_test[c].astype("string").fillna("__MISSING__")

na_cols = [c for c in X.columns if X[c].isna().any()]
for c in na_cols:
    X[c + "__isna"] = X[c].isna().astype(np.uint8)
    X_test[c + "__isna"] = X_test[c].isna().astype(np.uint8)

cat_cols2 = [c for c in X.columns if X[c].dtype == "string" or X[c].dtype == "object"]
cat_idx = [X.columns.get_loc(c) for c in cat_cols2]

# =========================
# Pseudo-labeling setup
# =========================
# Вариант 1: Используем ВСЕ pseudo-labels
pseudo_labels = pseudo_sub["target"].values
X_pseudo = X_test.copy()
y_pseudo = pseudo_labels

# Вариант 2: Только УВЕРЕННЫЕ предсказания (если у тебя есть вероятности)
# Если submission бинарный - используем всё
# Если есть soft predictions - фильтруем по confidence

print(f"Original train size: {len(X)}")
print(f"Pseudo-labeled test size: {len(X_pseudo)}")
print(f"Pseudo labels distribution: {pd.Series(y_pseudo).value_counts().to_dict()}")

# =========================
# Параметры
# =========================
best_params = dict(
    loss_function="Logloss",
    eval_metric="AUC",
    bootstrap_type="MVS",
    grow_policy="Lossguide",
    iterations=6740,
    learning_rate=0.035,
    depth=5,
    l2_leaf_reg=2.25,
    random_strength=4.8,
    border_count=205,
    min_data_in_leaf=61,
    rsm=0.95,
    one_hot_max_size=13,
    max_ctr_complexity=6,
    auto_class_weights="SqrtBalanced",
    max_leaves=52,
    subsample=0.7,
    verbose=200,
)

# =========================
# Training with Pseudo-labels
# =========================
seeds = [7896, 99999999, 568, 789 ]
all_oof = []
all_test_preds = []

# Разные стратегии pseudo-labeling
PSEUDO_WEIGHT = 0.5  # Вес pseudo-labels (можно тюнить: 0.3, 0.5, 1.0)

for seed in seeds:
    print(f"\n{'='*50}")
    print(f"Training with seed={seed} + Pseudo-labels")
    print(f"{'='*50}")
    
    best_params['random_seed'] = seed
    
    # CV только на ОРИГИНАЛЬНЫХ данных (важно!)
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    
    oof = np.zeros(len(X), dtype=float)
    test_pred = np.zeros(len(X_test), dtype=float)
    
    for fold, (tr_idx, va_idx) in enumerate(skf.split(X, y), 1):
        print(f"\n--- Seed {seed}, Fold {fold}/5 ---")
        
        # Оригинальные train данные этого фолда
        X_tr, y_tr = X.iloc[tr_idx], y[tr_idx]
        X_va, y_va = X.iloc[va_idx], y[va_idx]
        
        # Добавляем pseudo-labeled данные к train
        # Способ 1: Просто конкатенируем
        #X_tr_aug = pd.concat([X_tr, X_pseudo], ignore_index=True)
        #y_tr_aug = np.concatenate([y_tr, y_pseudo])
        
        # Способ 2: Sample pseudo с весом (чтобы не перегружать)
        n_pseudo_sample = int(len(X_tr) * PSEUDO_WEIGHT)
        pseudo_idx = np.random.RandomState(seed+fold).choice(len(X_pseudo), n_pseudo_sample, replace=False)
        X_tr_aug = pd.concat([X_tr, X_pseudo.iloc[pseudo_idx]], ignore_index=True)
        y_tr_aug = np.concatenate([y_tr, y_pseudo[pseudo_idx]])
        
        print(f"  Train size: {len(X_tr)} + {len(X_pseudo)} pseudo = {len(X_tr_aug)}")
        
        # Обновляем cat_idx для новых данных
        cat_idx_aug = [X_tr_aug.columns.get_loc(c) for c in cat_cols2]
        
        model = CatBoostClassifier(**best_params)
        model.fit(
            X_tr_aug, y_tr_aug,
            eval_set=(X_va, y_va),  # Валидация ТОЛЬКО на оригинальных!
            cat_features=cat_idx_aug,
            use_best_model=True,
            early_stopping_rounds=235,
        )
        
        oof[va_idx] = model.predict_proba(X_va)[:, 1]
        test_pred += model.predict_proba(X_test)[:, 1] / 5
    
    oof_auc = roc_auc_score(y, oof)
    print(f"\nSeed {seed} OOF AUC: {oof_auc:.6f}")
    
    all_oof.append(oof)
    all_test_preds.append(test_pred)

# =========================
# Усреднение
# =========================
print(f"\n{'='*50}")
print("RESULTS WITH PSEUDO-LABELING")
print(f"{'='*50}")

avg_oof = np.mean(all_oof, axis=0)
avg_oof_auc = roc_auc_score(y, avg_oof)
print(f"Average OOF AUC: {avg_oof_auc:.6f}")

avg_test_pred = np.mean(all_test_preds, axis=0)

# Порог
thr_grid = np.linspace(0.01, 0.99, 199)
best_thr, best_score = None, -1
for t in thr_grid:
    pred_bin = (avg_oof >= t).astype(int)
    s = roc_auc_score(y, pred_bin)
    if s > best_score:
        best_score, best_thr = s, t

print(f"Best threshold: {best_thr:.4f}")

# Submission
test_bin = (avg_test_pred >= best_thr).astype(int)
sub = pd.DataFrame({"id": test_id, "target": test_bin})
sub.to_csv("submission_pseudo.csv", index=False)

print(f"\nSubmission saved: submission_pseudo.csv")
print(f"Target distribution: {sub['target'].value_counts().to_dict()}")

Original train size: 151406
Pseudo-labeled test size: 22624
Pseudo labels distribution: {0: 17674, 1: 4950}

Training with seed=7896 + Pseudo-labels

--- Seed 7896, Fold 1/5 ---
  Train size: 121124 + 22624 pseudo = 143748
0:	test: 0.6274556	best: 0.6274556 (0)	total: 61.7ms	remaining: 6m 55s
200:	test: 0.7374509	best: 0.7374509 (200)	total: 18s	remaining: 9m 44s
400:	test: 0.7406457	best: 0.7406457 (400)	total: 34.2s	remaining: 9m
600:	test: 0.7434257	best: 0.7434614 (597)	total: 48s	remaining: 8m 10s
800:	test: 0.7436701	best: 0.7436701 (800)	total: 1m 8s	remaining: 8m 27s
1000:	test: 0.7438389	best: 0.7438830 (984)	total: 1m 22s	remaining: 7m 53s
1200:	test: 0.7436819	best: 0.7438980 (1004)	total: 1m 38s	remaining: 7m 32s
Stopped by overfitting detector  (235 iterations wait)

bestTest = 0.743897986
bestIteration = 1004

Shrink model to first 1005 iterations.

--- Seed 7896, Fold 2/5 ---
  Train size: 121125 + 22624 pseudo = 143749
0:	test: 0.6127245	best: 0.6127245 (0)	total: 83ms	